In [143]:
import pandas as pd

In [144]:
df_cnn=pd.read_csv('df_cnn_pruned_topic_combined_nov_21.csv')

In [145]:
df_cnn.head()

,Unnamed: 0.1,screenName,description,location,text,Unnamed: 0,topic
0,0,CNN,It’s our job to #GoThere & tell the most diffi...,NaN,Al Roker has revealed he has been recently hos...,0.0,health
1,1,CNN,It’s our job to #GoThere & tell the most diffi...,NaN,Uganda is grappling with its deadliest Ebola o...,2.0,health
2,2,CNN,It’s our job to #GoThere & tell the most diffi...,NaN,As some Chinese provinces mark 100 days of loc...,79.0,health
3,3,CNN,It’s our job to #GoThere & tell the most diffi...,NaN,When the US Food and Drug Administration confi...,155.0,health
4,4,CNN,It’s our job to #GoThere & tell the most diffi...,NaN,Cases of Covid-19 rebound following treatment ...,174.0,health


In [146]:
from sklearn.preprocessing import LabelEncoder

def encode_topic(dataframe):
    encoder = LabelEncoder()
    dataframe['topicEncoded'] = encoder.fit_transform(dataframe['topic'])
    dataframe['text'] = dataframe['text'].apply(lambda text: str(text).lower())
    return dataframe

In [147]:
df_cnn=encode_topic(df_cnn)

In [148]:
X = df_cnn['text']
y = df_cnn['topicEncoded']
from sklearn.model_selection import train_test_split
X_left_train, X_left_test , y_left_train, y_left_test = train_test_split(X, y , test_size = 0.20)

In [149]:
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
vocab = Counter()

In [150]:
from tensorflow.keras.preprocessing.text import Tokenizer
vocab_size = 5000
oov_token = "<OOV>"
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(X_left_train)

In [151]:
print(X_left_train)

922     inflation cools much more than expected in oct...
1013    mcdonald’s said this week that it sold half of...
906     tesla drivers will soon be able to take a vide...
834     brittney griner is set to appeal her verdict i...
772     herschel walker’s success in his upcoming runo...
                              ...                        
540     watch: former president obama mocked gop candi...
1327    here are five things you need to know today 👇\...
77      dr. rochelle walensky, director of the cdc, te...
486     the university of florida board of trustees un...
1010    "they can really negotiate right now" - bess f...
Name: text, Length: 1086, dtype: object


In [152]:
X_left_train_sequences = tokenizer.texts_to_sequences(X_left_train)
X_left_test_sequences = tokenizer.texts_to_sequences(X_left_test)

In [166]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_length = 100
padding_type = "post"
trunction_type="post"
X_left_train_padded = pad_sequences(X_left_train_sequences,maxlen=max_length, padding=padding_type,
                       truncating=trunction_type)
X_left_test_padded = pad_sequences(X_left_test_sequences,maxlen=max_length,
                               padding=padding_type, truncating=trunction_type)

In [167]:
!wget --no-check-certificate http://nlp.stanford.edu/data/glove.6B.zip -O /tmp/glove.6B.zip

In [168]:
import os
import zipfile
with zipfile.ZipFile('glove.6B.zip', 'r') as zip_ref:
    zip_ref.extractall('.//tmp/glove')

In [169]:
import numpy as np
embeddings_index = {}
f = open('./tmp/glove/glove.6B.100d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [173]:
embeddings_index.get("paper")

array([-0.85034  ,  0.33358  , -0.65889  , -0.49871  ,  0.36585  ,
       -0.19245  ,  0.25658  , -0.053408 ,  0.31474  ,  0.2443   ,
        0.29337  , -0.44917  ,  0.15175  ,  0.39314  , -0.31786  ,
        0.060525 ,  0.81775  , -0.38847  ,  0.76761  , -1.1041   ,
       -0.1544   ,  0.31655  , -0.37238  , -0.11485  ,  0.51635  ,
       -0.39289  ,  0.16301  , -0.2532   , -0.50976  ,  0.15201  ,
        0.27808  ,  0.52522  , -0.38815  , -0.3472   , -0.61818  ,
        0.17022  ,  0.12251  , -0.24191  , -0.38877  , -0.53176  ,
       -0.46987  , -0.70502  , -0.62126  , -0.38689  , -0.85637  ,
       -0.41003  , -0.47487  , -0.21083  , -0.81338  , -0.52398  ,
        0.49894  ,  0.37909  ,  0.55428  ,  1.123    , -0.42121  ,
       -1.5674   , -0.56892  ,  0.40819  ,  1.7949   ,  0.16856  ,
       -0.0029332,  0.28786  , -0.90088  , -0.094214 ,  0.79993  ,
       -0.39096  ,  0.76286  ,  0.71307  ,  0.13194  , -0.40756  ,
       -0.18687  ,  0.89562  ,  0.46867  , -0.0028801,  0.0253

In [174]:
from collections import Counter
vocab = Counter()
for index, row in df_cnn.iterrows():
    tokens = row['text'].split()
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    tokens = [word for word in tokens if len(word) > 1]
    vocab.update(tokens)

In [175]:
min_occurane = 2
tokens = [k for k,c in vocab.items() if c >= min_occurane]
print(len(tokens))

2984


In [176]:
word_index={}
for i in range(len(tokens)):
    word_index[tokens[i]]=i

print(len(word_index))

2984


In [177]:
embedding_matrix = np.zeros((len(word_index) + 1, max_length))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


In [178]:
from tensorflow.keras.layers import Dense, Embedding,GlobalMaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from keras.layers.convolutional import Conv1D

In [179]:
embedding_layer = Embedding(input_dim=len(word_index) + 1,
                            output_dim=max_length,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)

In [180]:
model = Sequential([
    embedding_layer,
  Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
  Dense(10, activation='relu'),
  Dense(1, activation='sigmoid')
])

In [181]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [182]:
print(y_left_train)

922     0
1013    0
906     0
834     4
772     4
       ..
540     3
1327    1
77      2
486     3
1010    0
Name: topicEncoded, Length: 1086, dtype: int32


In [183]:
history = model.fit(X_left_train_padded, y_left_train, epochs=20, validation_data=(X_left_test_padded, y_left_test))

Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node 'sequential_50/embedding_5/embedding_lookup' defined at (most recent call last):
    File "c:\Users\mmajeed2\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\mmajeed2\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\mmajeed2\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\mmajeed2\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
      app.start()
    File "c:\Users\mmajeed2\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\mmajeed2\AppData\Local\Programs\Python\Python310\l

In [130]:
loss, accuracy = model.evaluate(X_test_padded,y_test)
print('Testing Accuracy is {} '.format(accuracy*100))

20/20 [==============================] - 0s 4ms/step - loss: -1199804.3750 - accuracy: 0.2182
Testing Accuracy is 21.82103544473648 
